# Simulation of the wireless communication system project in Signals and transforms

In [15]:
import argparse
import numpy as np
from scipy import signal
import sounddevice as sd
import matplotlib.pyplot as plt
import wcslib as wcs

## Parameters

In [16]:
# # TODO: Add your parameters here. You might need to add other parameters as well.
# channel_id = 17
# Tb = 0.01
# fs = 4400
# dt = 1/22050
# Ac = 1
# bs = 1 / Tb
# #xb = wcs.encode_baseband_signal(bs, Tb, fs)
# #t = np.arange(0, xb.shape[0]) * (1/dt)
# # Message to be transmitted
# data = "Hello World!"
import numpy as np
import wcslib as wcs

# TODO: Add your parameters here. You might need to add other parameters as well.
channel_id = 17
Tb = 0.01
fs = 4400
dt = 1/22050
Ac = 1
fc = 4400
bs = 1 / Tb
wc = 27646    # rad/s
alpha = 2 * np.pi * 200
# Ar = |H(w)| * Ac
tr = 1
phi = -wc*tr

# Message to be transmitted
data = "Hello World!"

## Filter design

In [ ]:
# TODO: Design the filters

## Transmitter

In [17]:
parser = argparse.ArgumentParser(
    prog='transmitter',
    description='Acoustic wireless communication system -- transmitter.'
)
parser.add_argument(
    '-b',
    '--binary',
    help='message is a binary sequence',
    action='store_true'
)
parser.add_argument('message', help='message to transmit', nargs='?')
args = parser.parse_args()

if args.message is None:
    args.message = 'Hello World!'

# Set parameters
data = args.message

# Convert string to bit sequence
bs = wcs.encode_string(data)
    
# Encode baseband signal
xb = wcs.encode_baseband_signal(bs, Tb, fs)

# TODO: Put your transmitter code here (modulation, filtering, etc.)
t = np.arange(0, xb.shape[0]) * dt  
t1 = 10*Tb
# plot = int(np.ceil(t1/dt))

# Ensure the signal is mono, then play through speakers
xc = Ac * np.sin(wc * t)                            # Task 4b)
xm = xb * xc
xt = xm
yd = xm * np.sin(wc * t)

xt = np.stack((xt, np.zeros(xt.shape)), axis=1)
sd.play(xt, 1/dt, blocking=True)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10,15))

#### TASK 4A ####
plt.title("Task 4a")
ax1.plot(t, xb, label="xb(t)")
ax1.legend()
ax1.set_xlabel("t (s)")
ax1.set_ylabel("xb(t)")
ax1.set_xlim(0, t1)
ax1.grid()

#### TASK 4B ####
plt.title("Task 4b")
ax2.plot(t, xm, label="xm(t)")
ax2.legend()
ax2.set_xlabel("t (s)")
ax2.set_ylabel("xm(t)")
ax2.set_xlim(0, 4*Tb)
ax2.grid()

#### TASK 4C ####
plt.title("Task 4c")
ax3.plot(t, yd, label="yd(t)")
ax3.legend()
ax3.set_xlabel("t (s)")
ax3.set_ylabel("yd(t)")
ax3.set_xlim(0, t1)
ax3.grid()

fig.savefig('lab1_task4.png')

usage: transmitter [-h] [-b] [message]
transmitter: error: unrecognized arguments: --f=/Users/hannahpettersson/Library/Jupyter/runtime/kernel-v35dacfd6a21d6b3f6143fe1983e8d9fc2091ae5cc.json


SystemExit: 2

## Simulate transmission

In [18]:
yr = wcs.simulate_channel(xt, fs, channel_id)

NameError: name 'xt' is not defined

## Receiver

In [ ]:
# TODO: Add relevant parameters to parameters.py
from parameters import dt,Tb,fs,Ac, wc

# TODO: Put your receiver code (filtering, demodulation, etc.) here
# TODO: Implement demodulation, etc. here
# ...
num = [alpha**2]
den = [1, alpha, alpha**2]
H = signal.TransferFunction(num, den)

w, mag, phase = H.bode()
# mag_log = 20*np.log10(mag) # dB
phase_rad = (phase * np.pi) / 180

#### Task 4d) ####
fig, ax4 = plt.subplots(2, 1)
ax4[0].semilogx(w, mag)
# ax4[0].semilogx(w, mag_log)
ax4[0].set_ylabel("Magnitude (dB)")
ax4[0].grid()

ax4[1].semilogx(w, phase_rad)
ax4[1].set_ylabel("Phase (rad)")
ax4[1].set_xlabel("Frequency (rad/s)")
ax4[1].grid()
fig.savefig('lab1_task4d.png')
# plt.show()

    #### Task 4e) ####
__, yb, __ = signal.lsim(H, yd, t)

fig2, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,10))
plt.title("Task 4e")
ax1.plot(t, xb, label="xb(t)")
ax1.legend()
ax1.set_xlabel("t (s)")
ax1.set_ylabel("xb(t)")
ax1.set_xlim(0, t1)
ax1.grid()

ax2.plot(t, yb, label="yb(t)")
ax2.legend()
ax2.set_xlabel("t (s)")
ax2.set_ylabel("yb(t)")
ax2.set_xlim(0, t1)
ax2.grid()

fig2.savefig("lab1_task4e")

# Calculate complex baseband signal
# yb = ybi + 1j*ybq

# Symbol decoding
br = wcs.decode_baseband_signal(yb, Tb, fs)

data_rx = wcs.decode_string(br)
print('Received: ' + data_rx)